In [ ]:
import random
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
import time
from typing import List

def generate_points(length, height, num_points):
    points = []
    for i in range(num_points):
        x = random.uniform(0, length)
        y = random.uniform(0, height)
        points.append((x, y))

    sns.set_style('darkgrid')
    plt.figure(figsize=(6, 4))
    ax = sns.scatterplot(x=[p[0] for p in points], y=[p[1] for p in points])
    plt.title('Scatter Plot of Generated Points')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()

    #pairwise distances
    distances = []
    for i in range(num_points):
        row = []
        for j in range(num_points):
            dist = sqrt((points[i][0] - points[j][0]) ** 2 + (points[i][1] - points[j][1]) ** 2)
            row.append(dist)
        distances.append(row)

    return points, distances

length = 30.0
height = 12.0
num_points = 136
#num_points = 15

source_points, source_distances = generate_points(length, height, num_points)


In [ ]:
def calculate_length(route: List[int], distances: List[List[float]]) -> float:
    if len(route) != len(distances):
        raise ValueError("Route and distances lists must have the same length")
    
    length = 0
    for i in range(len(route) - 1):
        if route[i] < 0 or route[i] >= len(distances) or route[i+1] < 0 or route[i+1] >= len(distances):
            raise ValueError("Invalid index in route list")
        length += distances[route[i]][route[i+1]]
    length += distances[route[-1]][route[0]]
    return length


Nearest Neighbour

In [ ]:
import numpy as np
import itertools

def find_shortest_route(points):

    num_points = len(points)
    distances = [[0] * num_points for _ in range(num_points)]
    for i in range(num_points):
        for j in range(num_points):
            dist = sqrt((points[i][0] - points[j][0]) ** 2 + (points[i][1] - points[j][1]) ** 2)
            distances[i][j] = dist

    current_point = random.randint(0, num_points - 1)
    route = [current_point]

    while len(route) < num_points:
        nearest_neighbor = None
        nearest_distance = float('inf')
        for i in range(num_points):
            if i not in route and distances[current_point][i] < nearest_distance:
                nearest_neighbor = i
                nearest_distance = distances[current_point][i]
        
        route.append(nearest_neighbor)
        current_point = nearest_neighbor

    length = 0
    for i in range(num_points):
        length += distances[route[i]][route[(i + 1) % num_points]]

    return route, length

start_time = time.time()
shortest_route, shortest_length = find_shortest_route(source_points)
end_time = time.time()
bruteForceTime = end_time - start_time

2 Opt Algorithm

In [ ]:
from typing import List, Tuple
from math import sqrt
from random import randrange

def two_opt_algorithm(points: List[Tuple[float, float]], shortest_route: List[int], shortest_length: float, max_iterations: int=1000) -> Tuple[List[int], float]:

    point_dict = {point: i for i, point in enumerate(points)}
    indices = shortest_route.copy()

    def calculate_length(route: List[int]) -> float:
        length = 0
        for i in range(len(route) - 1):
            dist = sqrt((points[route[i]][0] - points[route[i+1]][0]) ** 2 + (points[route[i]][1] - points[route[i+1]][1]) ** 2)
            length += dist
        return length

    best_route = indices.copy()
    best_length = shortest_length
    for i in range(max_iterations):

        idx1, idx2 = sorted([randrange(len(points)) for _ in range(2)])
        new_route = indices[:idx1] + indices[idx1:idx2+1][::-1] + indices[idx2+1:]
        new_length = calculate_length(new_route)

        if new_length < best_length:
            best_route = new_route.copy()
            best_length = new_length

        indices = best_route.copy()

    optimized_route = [list(point_dict.keys())[list(point_dict.values()).index(idx)] for idx in best_route]
    optimized_route = [points.index(point) + 1 for point in optimized_route]

    return optimized_route, best_length

start_time = time.time()
twoopt_optimized_route, twoopt_optimized_length = two_opt_algorithm(source_points, shortest_route, shortest_length)
end_time = time.time()
twoOptTime = end_time - start_time


Ant colony Optimization

In [ ]:

import numpy as np
import random

def aco(distances, num_ants, num_iterations, decay, alpha=1, beta=1):
    num_points = len(distances)
    pheromone = np.ones((num_points, num_points))
    best_path = None
    best_distance = float('inf')
    for it in range(num_iterations):
        paths = []
        distances_ = []
        for ant in range(num_ants):
            current = random.randint(0, num_points - 1)
            path = [current]
            distance_ = 0
            visited = set()
            visited.add(current)
            unvisited = set(range(num_points))
            unvisited.remove(current)
            while unvisited:
                weights = [((pheromone[current][j])**alpha)*((1/distances[current][j])**beta) for j in unvisited]
                selected = random.choices(list(unvisited), weights=weights, k=1)[0]
                path.append(selected)
                visited.add(selected)
                unvisited.remove(selected)
                distance_ += distances[current][selected]
                current = selected
            paths.append(path)
            distances_.append(distance_)
        pheromone *= (1 - decay)
        for i in range(num_ants):
            distance_ = distances_[i]
            path = paths[i]
            for j in range(num_points - 1):
                x = path[j]
                y = path[j+1]
                pheromone[x][y] += 1/distance_
        idx = np.argmin(distances_)
        if distances_[idx] < best_distance:
            best_path = paths[idx]
            best_distance = distances_[idx]
    return best_path, best_distance

# example usage
num_ants = 100
num_iterations = 1000
decay = 0.1
alpha = 1
beta = 1

start_time = time.time()
ac_shortest_path, ac_distance = aco(source_distances, num_ants, num_iterations, decay, alpha, beta)
end_time = time.time()
acTime = end_time-start_time
#print("Best path found:", best_path)
#print("Distance of best path:", best_distance)


In [ ]:
import numpy as np

def nearest_neighbour_algorithm(dist_matrix):
    dist_matrix = np.array(dist_matrix)
    num_cities = dist_matrix.shape[0]
    visited_cities = np.zeros(num_cities, dtype=bool)    
    path = [0]
    visited_cities[0] = True
    total_distance = 0
    for i in range(num_cities-1):
        current_city = path[-1]
        nearest_neighbour_index = np.argmin(dist_matrix[current_city][visited_cities==False])
        nearest_neighbour = np.arange(num_cities)[visited_cities==False][nearest_neighbour_index]
        path.append(nearest_neighbour)
        visited_cities[nearest_neighbour] = True
        total_distance += dist_matrix[current_city][nearest_neighbour]
    # Add distance from last city to the starting city
    total_distance += dist_matrix[path[-1]][0]
    return path, total_distance

start_time = time.time()
nna_route, nna_length = nearest_neighbour_algorithm(source_distances)
end_time = time.time()
nnaTime = end_time-start_time


Greedy Algorithm

In [ ]:
import numpy as np

def greedy_algorithm(dist_matrix):
    dist_matrix = np.array(dist_matrix)
    num_cities = dist_matrix.shape[0]
    visited_cities = np.zeros(num_cities, dtype=bool)
    path = [0]
    visited_cities[0] = True
    total_distance = 0
    for i in range(num_cities-1):
        current_city = path[-1]
        nearest_neighbour_index = np.argmin(dist_matrix[current_city][visited_cities==False])
        nearest_neighbour = np.arange(num_cities)[visited_cities==False][nearest_neighbour_index]
        path.append(nearest_neighbour)
        visited_cities[nearest_neighbour] = True
        total_distance += dist_matrix[current_city][nearest_neighbour]
    # Add distance from last city to the starting city
    total_distance += dist_matrix[path[-1]][0]
    return path, total_distance

start_time = time.time()
greedy_path, greedy_distance = greedy_algorithm(source_distances)
end_time = time.time()
greedyTime = end_time-start_time


In [ ]:
import pandas as pd

'''
outputDataframe = {

    "Algorithm"         : ["Brute force", "Two Opt Algorithm", "Three Opt Algorithm", "Genetic Algorithm", "Ant Colony Optimization", "Particle Swarm Optimization", "Lin Kernighan", "Simulated Annealing"] ,
    "Distance"          : [shortest_length, twoopt_optimized_length, threeopt_optimized_length, ga_distance, ac_distance, pso_length, linKernighan_length, simAnnealing_length],
    "Calculation Time"  : [bruteForceTime, twoOptTime, threeOptTime, gaTime, acTime, psoTime, linKernighanTime, simAnnealingTime]

}
'''

outputDataframe = {

    "Algorithm"         : ["Greedy Algorithm", "Nearest Neighbour", "Two Opt Algorithm","Ant Colony Optimization"],
    "Distance"          : [greedy_distance, shortest_length, twoopt_optimized_length, ac_distance],
    "Calculation Time"  : [greedyTime, bruteForceTime, twoOptTime, acTime]

}

df = pd.DataFrame(outputDataframe)
print(df)

In [ ]:
sumTime = bruteForceTime + twoOptTime +  acTime + greedyTime
print(sumTime)